In [1]:
import re
import wikipediaapi

In [2]:
import stanza

In [21]:
import random

In [174]:
import json

In [3]:
nlp = stanza.Pipeline(
    "hu",
    processors="tokenize",
    tokenize_no_ssplit=False,
    use_gpu=False
)

2026-01-15 14:53:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2026-01-15 14:53:02 INFO: Downloaded file to C:\Users\fegyo\stanza_resources\resources.json
2026-01-15 14:53:02 INFO: Loading these models for language: hu (Hungarian):
| Processor | Package |
-----------------------
| tokenize  | szeged  |

2026-01-15 14:53:02 INFO: Using device: cpu
2026-01-15 14:53:02 INFO: Loading: tokenize
2026-01-15 14:53:04 INFO: Done loading processors!


In [4]:
wiki_hu = wikipediaapi.Wikipedia(
    user_agent="MaturaDatasetBot/1.0 (fegyo.benedek@gmail.com)",
    language='hu',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)

In [134]:
l = [1,2,3,4,5]
random.shuffle(l)
print(l)

[4, 5, 2, 3, 1]


In [186]:
def crawler(root, limit=10, depth=3, output=None):
    if output is None:
        output = []

    if depth <= 0:
        return output

    root_cat = wiki_hu.page(root)
    members = list(root_cat.categorymembers.values())
    random.shuffle(members)
    for page in members:
        if (page.ns != wikipediaapi.Namespace.CATEGORY):
            # Longer than a 1000 characters
            if len(page.text) > 1000:
                paragraphs = page.text.split("\n\n")
                if len(paragraphs) > 10:
                    output.append([page.title, paragraphs])
        elif depth > 1: 
            print(f"Now exploring {page.title} at depth {depth}")
            crawler(page.title, limit=limit, depth=depth-1, output=output)

        if len(output) % 50 == 0:
            print(f"Collected {len(output)} articles so far...")
        if len(output) >= limit:
            break

    return output


In [ ]:
def create_dataset(name, root, limit, depth):
    # Crawl dataset
    output = crawler(root, limit=limit, depth=depth)
    clean_output = []
    for i in range(len(output)):
        title, paragraphs = output[i]
        clean_paragraphs = []
        for para in paragraphs:
            t_para = para.split("\n")[0]
            if t_para in ['Megjegyzések','Források', 'Irodalom', 'Külső hivatkozások', 'Lásd még', 'Bibliográfia', 'Hivatkozások', "Jegyzetek","== Jegyzetek =="]:
                break
            clean_paragraphs.append(para)
        clean_output.append([title, clean_paragraphs])
        
    # Segment into two paragraph pars [] ... []
    id = 0
    segmented_output = {}
    for title, paragraphs in clean_output:
        p = paragraphs
        random.shuffle(p)
        if len(p) < 6:
            continue
        else:    
            t_1 = p[1] + " ... " + p[2]
            t_2 = p[3] + " ... " + p[4]
            t_3 = p[5] + " ... " + p[6]
            segmented_output[id] = t_1
            segmented_output[id+1] = t_2
            segmented_output[id+2] = t_3
            id += 3
        
    with open(f'{name}.json', 'w', encoding='utf-8') as f:
        json.dump(segmented_output, f, ensure_ascii=False, indent=4)

In [191]:
#"Kategória:Történelem"
create_dataset("tortenelem", "Kategória:Történelem", limit=500, depth=2)

Collected 0 articles so far...
Now exploring Kategória:Naptárak at depth 2
Collected 0 articles so far...
Collected 0 articles so far...
Collected 0 articles so far...
Collected 0 articles so far...
Collected 0 articles so far...
Now exploring Kategória:Művészettörténet at depth 2
Now exploring Kategória:Történelemmel kapcsolatos portálok at depth 2
Now exploring Kategória:Történelmi népek at depth 2
Now exploring Kategória:Hadviselés at depth 2
Now exploring Kategória:Gyarmatosítás at depth 2
Collected 50 articles so far...
Now exploring Kategória:Történelmi témájú listák at depth 2
Now exploring Kategória:Évek at depth 2
Now exploring Kategória:Történeti földrajz at depth 2
Now exploring Kategória:Történelmi rekordok at depth 2
Now exploring Kategória:Történelmi események at depth 2
Now exploring Kategória:Korábbi közigazgatási egységek országok szerint at depth 2
Now exploring Kategória:Technikatörténet at depth 2
Collected 100 articles so far...
Now exploring Kategória:Nem megvalós

In [193]:
#"Kategória:Magyarország történelme"
create_dataset("magyar_tortenelem","Kategória:Magyarország történelme", limit=600, depth=2)

Collected 0 articles so far...
Collected 0 articles so far...
Now exploring Kategória:Magyarország háborúi at depth 2
Collected 0 articles so far...
Collected 0 articles so far...
Collected 0 articles so far...
Collected 0 articles so far...
Collected 0 articles so far...
Now exploring Kategória:Magyar gazdaságtörténet at depth 2
Now exploring Kategória:Magyar nemzeti jelképek at depth 2
Now exploring Kategória:Magyarország történelmi tájai at depth 2
Now exploring Kategória:Magyar történelmi államszervezet at depth 2
Now exploring Kategória:Magyar történetírás at depth 2
Now exploring Kategória:Magyarország történelme megyék szerint at depth 2
Now exploring Kategória:Habsburg-ellenes felkelések at depth 2
Now exploring Kategória:Csonkok (magyar történelem) at depth 2
Now exploring Kategória:Magyar települések történelme at depth 2
Collected 50 articles so far...
Now exploring Kategória:Magyar társadalomtörténet at depth 2
Now exploring Kategória:Magyar történelmi témájú listák at dept